## Conjunto de datos Bricks

Lo primero que deberemos hacer es descargar los datos paara el entrenamiento. Utilizaremos el conjunto de datos [Images of LEGO Bricks](https://www.kaggle.com/datasets/joosthazelzet/lego-brick-images), disponible en Kaggle. Se trata de una colección de 40.000 fotografias informatizadas de 50 ladrillos de juguete distintos, tomadas desde distintos ángulo.

Utilizamos la función de Keras `image_dataset_from_directory` para crear un TensorFlow Dataset. De este modo podemos leer grupos de imágenes en la memoria solo cuando sea necesario (en el entrenamiento). Tambien redimensionamos las imagenes a un tamaño de 64 x 64.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

print(f'Versión de TensorFlow: {tf.__version__}')
print(f"Dispositivo de entrenamiento: {tf.config.list_physical_devices('GPU')}")

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    "../datasets/lego_bricks",
    labels=None,
    color_mode='grayscale',
    image_size=(64, 64),
    batch_size=128,
    shuffle=True,
    seed=42,
    interpolation='bilinear'
)

Los datos originales se dimensionan en el rango [0, 255] para indicar la intensidad del pixel. Al entrenar redes GAN ajustamos los datos al rango [-1, 1], de forma que podamos usar la función de activación tanh (tangente hiperbólica) en la capa final del generador, que tiende a ofrecer gradientes más fuertes que la función sigmoide. 

In [ ]:
def preprocess(img):
    return (tf.cast(img, "float32") - 127.5) / 127.5

train = train_data.map(lambda x: preprocess(x))

## Discriminador
El objetivo del discriminador es predecir si una imagen es real o falsa. Es un problema de clasificación de imagen supervisada.

Conviene observar que utilizamos un stride de 2 en algunas de las capas Conv2D para reducir la forma espacial del tensor cuando pasa por la red, aumentando al mismo tiempo el numero de canales. Usamos la función de activación *sigmoid* en la capa Conv2D final para producir un número entre 0 y 1.

In [ ]:
discriminator_input = tf.keras.layers.Input(shape=(64, 64, 1))

# Bloque 01
x = tf.keras.layers.Conv2D(64, kernel_size=4, strides=2, padding='same', use_bias=False)(discriminator_input)
x = tf.keras.layers.LeakyReLU(0.2)(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Bloque 02
x = tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Bloque 03
x = tf.keras.layers.Conv2D(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Bloque 04
x = tf.keras.layers.Conv2D(512, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Bloque 05
x = tf.keras.layers.Conv2D(1, kernel_size=4, strides=1, padding='valid', use_bias=False, activation='sigmoid')(x)

discrimnator_output = tf.keras.layers.Flatten()(x)

discriminator = tf.keras.models.Model(discriminator_input, discrimnator_output)

## Generador

La entrada para el generador será un vector tomado de una distribución normal estandar multivariada. El resultado o salida es una imagen que tiene el mismo tamaño que una imagen de los datos de entrenamiento originales.

In [ ]:
generator_input = tf.keras.layers.Input(shape=(100, ))

# Bloque 01
x = tf.keras.layers.Reshape((1, 1, 100))(generator_input)
x = tf.keras.layers.Conv2DTranspose(512, kernel_size=4, strides=1, padding='valid', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)

# Bloque 02
x = tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)

# Bloque 03
x = tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)

# Bloque 04
x = tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
x = tf.keras.layers.LeakyReLU(0.2)(x)

generator_output = tf.keras.layers.Conv2DTranspose(1, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh')(x)

generator = tf.keras.models.Model(generator_input, generator_output)

### Stride

Resumidamente, lo que hace el parametro *stride* es "cortar" las dimensiones de la entrada. Por ejemplo, si tenemos una imagen de 128 x 128 y aplicamos un *stride* de 2 obtendremos una imagen de 64 x 64. En el caso de usar una capa convolucional transpuesta el efecto será el contrario, ampliaremos el tamaño de la imagen en 2x.

La información se ha sacado de esta [respuesta de StackOverflow](https://stackoverflow.com/questions/65440993/how-strides-effect-input-shapes-in-keras) 

### Upsampling2D vs Conv2DTranspose

Se pueden usar las capas Upsampling2D en vez de las Conv2DTranspose. Por ejemplo:

``` python
x = tf.keras.layers.Upsampling2D(size=2)(x)
x = tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=1, padding='same')(x)
```

La capa Upsampling2D repite cada fila y cada columna de su entrada para duplicar el tamaño. La capa Conv2D con *stride* 1 realiza la operación de convolución. Es una idea similar a la transposición convolucional, pero en lugar de rellenar los espacios entre pixeles con ceros el *upsampling* tan solo repite los valores de pixel existentes.

Se ha demostrado que Conv2DTranspose puede producir artefactos. Lo ideal sería probar ambos métodos y verificar cual produce mejores resultados para según que objetivo.

#### Enlaces interesantes

- [An Introduction to different Types of Convolutions in Deep Learning](https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d)

## Entrenar la red DCGAN

Podemos entrenar al discriminador creando un conjunto de entrenamiento, en el que algunas de las imágenes son observaciones reales del conjunto y otras son resultados falsos del generador. Después tratamos esto como un problema de aprendizaje supervisado, donde las etiquetas son 1 para las imagenes reales y 0 para las falsas, con entropía cruzada binaria como función de pérdida.

Para entrenar al generador debemos encontrar una forma de puntuar cada imagen generada, de forma que pueda optimizar hacia imágenes con puntuación alta. Podemos generar un grupo de imagenes y pasarlas por el discriminador para obtener una puntuación por cada imagen. La función de pérdida del generador es simplemente la entropia cruzada binaria entre estas probabilidades y un vector de unos, porque queremos entrenar al generador para producir imágenes que el discriminador piensa que son reales.

In [ ]:
class DCGAN(tf.keras.models.Model):
    def __init__(self, discriminator, generator, latent_dim) -> None:
        super(DCGAN, self).__init__()
        self.discriminator = discriminator 
        self.generator = generator 
        self.latent_dim = latent_dim 
        
    def compile(self, discriminator_optimizer, generator_optimizer):
        super(DCGAN, self).compile()
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()
        self.discriminator_optimizer = discriminator_optimizer
        self.generator_optimizer = generator_optimizer
        self.discriminator_loss_metric = tf.keras.metrics.Mean(name = 'discriminator_loss')
        self.generator_loss_metric = tf.keras.metrics.Mean(name = 'generator_loss')
        
    @property
    def metrics(self):
        return [self.discriminator_loss_metric, self.generator_loss_metric]
    
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: 
            generated_images = self.generator(random_latent_vectors, training=True)
            
            real_predictions = self.discriminator(real_images, training=True) 
            fake_predictions = self.discriminator(generated_images, training=True) 
            
            real_labels = tf.ones_like(real_predictions)
            real_noisy_labels = real_labels + 0.1 * tf.random.uniform(tf.shape(real_predictions))
            
            fake_labels = tf.zeros_like(fake_predictions)
            fake_noisy_labels = fake_labels - 0.1 * tf.random.uniform(tf.shape(fake_predictions))
            
            discriminator_real_loss = self.loss_fn(real_noisy_labels, real_predictions)
            discriminator_fake_loss = self.loss_fn(fake_noisy_labels, fake_predictions)
            discriminator_loss = (discriminator_real_loss + discriminator_fake_loss) / 2.0
            
            generator_loss = self.loss_fn(real_labels, fake_predictions)
        
        gradients_of_discriminator = disc_tape.gradient(discriminator_loss, self.discriminator.trainable_variables)
        gradients_of_generator = gen_tape.gradient(generator_loss, self.generator.trainable_variables)
        
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        
        self.discriminator_loss_metric.update_state(discriminator_loss)
        self.generator_loss_metric.update_state(generator_loss)
        
        return {m.name: m.result() for m in self.metrics}
            

In [ ]:
dcgan = DCGAN(discriminator=discriminator, generator=generator, latent_dim=100)
dcgan.compile(
    discriminator_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999),
    generator_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)
)
dcgan.fit(train, epochs=100)

In [ ]:
def compare_images(img1, img2):
    return np.mean(np.abs(img1 - img2))